In [2]:
import numpy as np
import matplotlib.pylab as plt
np.set_printoptions(precision=2)

# Přednáška 6: Stavové modely, Kalmanův filtr

**Obsah přednášky:**
- motivační příklad
- stavové modely
- Kalmanův filtr
- Rozšíření KF (EKF...)

## Motivační příklad

Uvažujme, že pozorujeme těleso vržené vzhůru a zajímá nás jeho rychlost $v_t$ a "přesná" výška $h_t$, které budeme nazývat **stavovými proměnnými** (nebo **stavem**). $t=1,2,\ldots$ jsou diskrétní časové okamžiky (např. sekundy). K dispozici máme nepřesná (tedy zašuměná) **měření** výšky získaná z GPS, tato měření označme $y_{h,t}$. Pro jednoduchost zanedbáme vliv odporových sil.

Ze středoškolské fyziky (a pro přítomnost šumu i statistiky) víme, že pro výšku $h_t$ a rychlost $v_t$ v čase $t$ platí

$$
\begin{aligned}
h_t &= h_{0} + v_0 \Delta_t - \frac{1}{2} g \Delta_t^2 + w_{h,t}, \\
v_t &= h_0 - g\Delta_t + w_{v,t},
\end{aligned}
$$

kde $g = 9.81ms^{-2}$ je gravitační zrachlení, $v_0$ je počáteční rychlost, $y_0$ je počáteční výška a $\Delta_t = t-t_0$ je doba od počátečního času $t_0$. Zpravidla $t_0 = 0$, čímž se z $\Delta_t$ stane jen $t$ a dostaneme vzorečky na něž jsme byli na střední škole zvyklí. Členy $w_{h,t}$ a $w_{v,t}$ představují šum na přítomných stavových veličinách.

Všimněme si, že **stav** je vlastně jakýsi **parametr** celého tohoto modelu, jenže narozdíl od klasického parametru se nám vyvíjí. Rychlost tělesa vrženého vzhůru bude klesat, zatímco výška bude stoupat.

V předmětu jsme si zvykli na sekvenční (online) modelování. Protože bychom chtěli podobně přistoupit k $h_t$ a $v_t$, jež se nám ale v čase vyvíjí, potřebovali bychom model trochu upravit, aby tento časový vývoj z času $t-1$ na čas $t$ zohlednil. Model si tedy přepíšeme:

$$
\begin{aligned}
h_t &= h_{t-1} + v_{t-1} \Delta_t - \frac{1}{2} g \Delta_t^2,\\
v_t &= v_{t-1} - g \Delta_t.
\end{aligned}
$$

Chceme-li modelovat více veličin současně, přepíšeme model do maticové podoby:

$$
\underbrace{
\begin{bmatrix}
h_t \\
v_t
\end{bmatrix}
}_{x_t}
=
\underbrace{
\begin{bmatrix}
1 & \Delta_t \\
0 & 1
\end{bmatrix}
}_{A_t}
\underbrace{
\begin{bmatrix}
h_{t-1} \\
v_{t-1}
\end{bmatrix}
}_{x_{t-1}}
+
\underbrace{
\begin{bmatrix}
- \frac{1}{2}\Delta_t \\
- \Delta_t
\end{bmatrix}
}_{B_t}
\underbrace{
g
}_{u_t}
+
\underbrace{
\begin{bmatrix}
w_{h,t} \\
w_{v,t}
\end{bmatrix}
}_{w_t}.
$$

Připomeňme, že máme i zašuměná měření výšky z GPS, označená jako $y_{h,t}$. Ta zapíšeme jednoduše

$$
y_{h,t} = h_t + v_t.
$$

Jelikož máme ale vektor stavů, můžeme použít ekvivalentní zápis

$$
\underbrace{
\begin{bmatrix}
y_{h,t} \\
0
\end{bmatrix}
}_{y_t}
=
\underbrace{
\begin{bmatrix}
1 & 0 \\
0 & 0 \\
\end{bmatrix}
}_{H_t}
x_t
+ v_t.
$$

### Výsledný stavový model:
$$
\begin{aligned}
\begin{bmatrix}
h_t \\
v_t
\end{bmatrix}
&=
\begin{bmatrix}
1 & \Delta_t \\
0 & 1
\end{bmatrix}
\begin{bmatrix}
h_{t-1} \\
v_{t-1}
\end{bmatrix}
+
\begin{bmatrix}
- \frac{1}{2}\Delta_t \\
- \Delta_t
\end{bmatrix}
g
+
\begin{bmatrix}
w_{h,t} \\
w_{v,t}
\end{bmatrix}
\\
\begin{bmatrix}
y_{h,t} \\
0
\end{bmatrix}
&=
\begin{bmatrix}
1 & 0 \\
0 & 0 \\
\end{bmatrix}
x_t
+ v_t.
\end{aligned}
$$

---

>**OBECNĚ**
>$$
\begin{aligned}
x_t &= A_t x_{t-1} + B_t u_t + w_t, \\
y_t &= H_t x_t + v_t,
\end{aligned}
$$
>
> kde $x_t$ je **stav**, $u_t$ je **řídící veličina**, $w_t$ a $v_t$ jsou **šum stavu** a **šum měření** a $A_t, B_t$ a $H_t$ jsou matice patřičných rozměrů.

---

# Stavový model

Stavový model nějakého systému nebo procesu je charakterizován třemi veličinami:
- **stavem**, označeným zpravidla $x_t$, který nemůžeme pozorovat, ale můžeme jej odhadovat,
- **vstupem**, též nazývaným **řídící veličinou** $u_t$, kterou známe,
- **výstupem**, výše označeným $y_t$, což pozorovatelná veličina determinovaná stavovou a řídící veličinou $x_t$ a $u_t$.

Stavový model může být v čase buď diskrétní, tj. popsaný v časových okamžicích $t=1,2,\ldots$, nebo spojitý, pak je popsaný derivacemi. My se budeme striktně držet diskrétního stavu.

Obecně zapisujeme stavový model jako známé funkce stavů a vstupů, přičemž tyto funkce mohou a nemusí být v čase konstantní,

$$
\begin{aligned}
x_t &= f_t(x_{t-1}, u_t), \\
y_t &= g_t(x_t).
\end{aligned}
$$

V teorii řízení je často vstupní (řídící) veličina i ve funkci $g_t$, my pro jednoduchost budeme uvažovat uvedený případ. Je-li systém lineární, potom - jak už bylo uvedeno výše - jej můžeme zapsat

>$$
\begin{aligned}
x_t &= A_t x_{t-1} + B_t u_t + w_t, \\
y_t &= H_t x_t + v_t,
\end{aligned}
$$

kde $x_t$ je **stav**, $u_t$ je **řídící veličina**, $w_t$ a $v_t$ jsou **šum stavu** a **šum měření** a $A_t, B_t$ a $H_t$ jsou matice patřičných rozměrů. Pokud je systém časově invariantní, jsou matice konstantní, $A_t=A, B_t=B, H_t=H$.

## Markovská vlastnost, skrytý markovský model (Hidden Markov Models, HMM)

V souvislosti se stavovými modely se často mluví o markovské vlastnosti. Markovský proces, též markovský model 1. řádu, je model, v němž aktuální stav závisí pouze na stavu předchozím. Je tedy popsán pravděpodobnostmi

$$
\begin{aligned}
&p(x_t|x_{t-1}) &\text{(pravděpodobnost přechodu)}, \\
&p(x_0) &\text{(počáteční stav)}.
\end{aligned}
$$

Pokud bychom pracovali s modelem vyššího řádu, např. $n$-tého, byla by pravděpodobnost přechodu $p(x_t|x_{t-1},...,x_{t-n})$. To ovšem vede k náročnějším výpočtům.

Pokud není markovský proces přímo pozorovatelný, ale lze na něj nahlížet prostřednictvím jiné pozorovatelné veličiny $y_t$, mluvíme o **skrytém markovském procesu**. My budeme navíc předpokládat, že celý proces je *spojitý*.

![HMM](img/hmm.png)

**Úkoly:**
- najdete ve stavovém modelu markovskou vlastnost?
- rozpoznáte ve stavovém modelu HMM?

# Kalmanův filtr
- Kalman, Bucy, Stratonovič
- Apollo...
![Kalman](img/Rudolf_Kalman.jpg)


## Odvození
Vraťme se ke stavovému modelu (pro jednoduchost časově invariantnímu) zadanému systémem rovnic

$$
\begin{aligned}
x_t &= A x_{t-1} + B u_t + w_t, \\
y_t &= H x_t + v_t,
\end{aligned}
$$

kde obě šumové proměnné jsou nezávislé normální a centrované v nule

$$
\begin{aligned}
w_t &\sim \mathcal{N}(0, Q),\\
v_t &\sim \mathcal{N}(0, R).
\end{aligned}
$$

Z toho vidíme, že
$$
\begin{aligned}
x_t &\sim \mathcal{N}(Ax_{t-1} + Bu_t, Q),\\
y_t &\sim \mathcal{N}(Hx_{t}, R).
\end{aligned}
$$

Jelikož hodláme k filtraci (tj. sekvenčnímu odhadu) stavu $x_t$ přistoupit bayesovsky, potřebujeme ještě apriorní distribuci pro $x_t$. Model $y_t$ je normální, konjugované apriorno bude tedy rovněž normální se střední hodnotou označenou $x_{t-1}^{+}$ a kovarianční maticí $x_{t-1}^{+}$,

$$
\pi(x_t|y_{0:t-1}, y_{0:t-1}, u_{0:t-1}) = \mathcal{N}(x_{t-1}^{+}, x_{t-1}^{+}).
$$

Kalmanův filtr běží ve dvou krocích - **predikce stavu** a **update stavu** (též korekce).

### Predikce

$$
\pi(x_{t}|y_{0:t-1}, x_{0:t}, u_{0:t})
= \int \pi(x_{t}|x_{t-1}, u_{t}) \pi(x_{t-1}|y_{0:t-1}, x_{0:t-1}, u_{0:t-1}) dx_{t-1}.
$$

Jelikož násobíme dvě normální distribuce a marginalizujeme, dostaneme opět normální distribuci
$\mathcal{N}(x_{t}^{-}, P_{t}^{-})$ s hyperparametry

$$
\begin{aligned}
    x_{t}^{-} &= A x_{t-1}^{+} + B u_{t}, \\
    P_{t}^{-} &= A P_{t-1}^{+} A^{\intercal} + Q.
\end{aligned}
$$

Vidíme, že násobení dvou normálních distribucí a následná marginalizace vlastně jen vyčíslí rovnici pro stav.

### Update
$$
\pi(x_{t}|y_{0:t}, x_{0:t}, u_{0:t}) 
\propto
\pi(x_{t}|y_{0:t-1},x_{0:t}, u_{0:t}) f(y_{t}|x_{t})
$$

Už v minulosti (v přednášce o regresi) se nám osvědčilo přepsat si model do tvaru distribuce z exponenciální třídy a k tomu vhodně napsat i apriorno. Potom je bayesovský update jen součet hyperparametru a suficientní statistiky. Model bude vypadat následovně:

$$
\begin{aligned}
    f(y_{t}|x_{t})
    &\propto \exp
        \left\{ 
            -\frac{1}{2} 
            (y_{t} - H x_{t})^{\intercal}
            R^{-1}
            (y_{t} - H x_{t})
        \right\} \\
    &= \exp
        \Bigg\{ 
            \operatorname{Tr}
            \bigg(
            \underbrace{
                -\frac{1}{2}
                \begin{bmatrix}
                    -1 \\ x_{t}
                \end{bmatrix}
                \begin{bmatrix}
                    -1 \\ x_{t}
                \end{bmatrix}^{\intercal}
            }_{\eta}
            \underbrace{
                \begin{bmatrix}
                    y_{t}^{\intercal} \\ H^{\intercal}
                \end{bmatrix}
                R^{-1}
                \begin{bmatrix}
                    y_{t}^{\intercal} \\ H^{\intercal}
                \end{bmatrix}^{\intercal}
            }_{T(y_{t})}
            \bigg)
        \Bigg\}.
\end{aligned}
$$

Konjugovaný tvar apriorna má podobu

$$
\begin{aligned}
    \pi(x_{t}|y_{0:t-1},x_{0:t}, u_{0:t}) 
    &\propto \exp
        \left\{ 
            -\frac{1}{2} 
            (x_{t} - x_{t}^{-})^{\intercal}
            (P_{t}^{-})^{-1}
            (x_{t} - x_{t}^{-})
        \right\} \notag \\
    &= \exp
        \Bigg\{
            \operatorname{Tr}
            \bigg(
            \underbrace{
                -\frac{1}{2}
                \begin{bmatrix}
                    -1 \\ x_{t}
                \end{bmatrix}
                \begin{bmatrix}
                    -1 \\ x_{t}
                \end{bmatrix}^{\intercal}
            }_{\eta}
            \underbrace{
                \begin{bmatrix}
                    (x_{t}^{-})^{\intercal} \\ I
                \end{bmatrix}
                (P_{t}^{-})^{-1}
                \begin{bmatrix}
                    (x_{t}^{-})^{\intercal} \\ I
                \end{bmatrix}^{\intercal}
            }_{\xi_{t}}
            \bigg)
        \Bigg\},
        \notag
\end{aligned}
$$
kde $I$ je jednotková matice odpovídajícího rozměru.

Bayesovský update je součet hyperparametru a suficientní statistiky,

$$
\begin{aligned}
    \xi_{t} 
    &= \xi_{t-1} +  T(y_{t})  \notag \\
    &=
    \begin{bmatrix}
        (x_{t}^{-})^{\intercal} (P_{t}^{-})^{-1} x_{t}^{-} + y_{t}^{\intercal} R^{-1} y_{t}, 
            & (x_{t}^{-})^{\intercal} (P_{t}^{-})^{-1} + y_{t}^{\intercal} R^{-1} H \\
        (P_{t}^{-})^{-1} (x_{t}^{-})^{\intercal} + H^{\intercal} R^{-1} y_{t}, 
        & (P_{t}^{-})^{-1} + H^{\intercal} R^{-1} H.
    \end{bmatrix}
\end{aligned}
$$

Aposteriorní parametry normálního rozdělení odvodíme snadno,

$$
\begin{aligned}
    P_{t}^{+} &= (\xi_{t;[2,2]})^{-1} \\
              &= \left[ (P_{t}^{-})^{-1} + H^{\intercal} R^{-1} H \right]^{-1} \\
              &= (I - K_{t} H) P_{t}^{-} \\
    x_{t}^{+} &= (\xi_{t;[2,2]})^{-1} \xi_{t;[2,1]} \\
              &= P_{t}^{+} \left[ (P_{t}^{-})^{-1} (x_{t}^{-})^{\intercal} + H^{\intercal} R^{-1} y_{t}\right]  \\
              &= x_{t}^{-} + P_{t}^{+} H^{\intercal} R^{-1}(y_{t} - Hx_{t}^{-})
\end{aligned}
$$

kde

$$
   K_{t} = P_{t}^{-} H^{\intercal}(R + H P_{t}^{-}H^{\intercal})
$$

je Kalmanovo zesílení. Konkrétně uvedená podoba je *optimální Kalmanovo zesílení*, neboť minimalizuje střední kvadratickou chybu. Obecně čím větší zesílení, tím větší důraz nová měření. Filtr je pak citlivější, ale hůře filtruje šum.

Není bez zajímavosti, že poslední výrazy pro $x_t^{+}, P_t^{+}$ a $K_t$ nejsou bayesovskou specialitou, ale vzniknou i nebayesovským odvozením. Ostatně bayesovský pohled na Kalmanův filtr vznikl výrazně později, než filtr samotný.